In [1]:
import pandas as pd
import numpy as np
import py_pcha
from py_pcha import PCHA
import matplotlib.pyplot as plt
from numpy.matlib import repmat

In [2]:
data=pd.read_csv("/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/us_states_covid19_daily.csv")
data

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201206,AK,35720.0,NaN,1042056.0,NaN,totalTestsViral,1077776.0,164.0,799.0,...,1077776,0,0,7b1d31e2756687bb9259b29195f1db6cdb321ea6,0,0,0,0,0,NaN
1,20201206,AL,269877.0,45962.0,1421126.0,NaN,totalTestsPeopleViral,1645041.0,1927.0,26331.0,...,1691003,12,0,19454ed8fe28fc0a7948fc0771b2f3c846c1c92e,0,0,0,0,0,NaN
2,20201206,AR,170924.0,22753.0,1614979.0,NaN,totalTestsViral,1763150.0,1076.0,9401.0,...,1785903,40,21,25fc83bffff5b32ba1a737be8e087fad9f4fde33,0,0,0,0,0,NaN
3,20201206,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,8c39eec317586b0c34fc2903e6a3891ecb00469e,0,0,0,0,0,NaN
4,20201206,AZ,364276.0,12590.0,2018813.0,NaN,totalTestsPeopleViral,2370499.0,2977.0,28248.0,...,2383089,25,242,7cf59da9e4bc31d905e179211313d08879880a85,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15628,20200124,WA,0.0,NaN,0.0,NaN,totalTestEncountersViral,0.0,NaN,NaN,...,0,0,0,82726df68eb97c98a4a6ee792349e547023147d5,0,0,0,0,0,NaN
15629,20200123,MA,NaN,NaN,NaN,NaN,totalTestsViral,2.0,NaN,NaN,...,0,0,0,76bc987d054b119a4e05a4a43742249c0b0568b6,0,0,0,0,0,NaN
15630,20200123,WA,0.0,NaN,0.0,NaN,totalTestEncountersViral,0.0,NaN,NaN,...,0,0,0,1c2229c239ffad5e5fdd9d76c641dc9166caf6ba,0,0,0,0,0,NaN
15631,20200122,MA,NaN,NaN,NaN,NaN,totalTestsViral,1.0,NaN,NaN,...,0,0,0,01f5dcd6631859503ef1b62d81d49e41d12fc1bd,0,0,0,0,0,NaN


In [ ]:
data=pd.read_csv("/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/us_states_covid19_daily.csv")
data

Data for 50 states, DC, and 5 territories on December 6, 2020. Select states and territories with missing values were removed. 

In [3]:
dec6 = data.loc[data['date'] == 20201206]
dec6 = dec6[['positive','negative','totalTestResults','recovered', 'death']]
dec6 = dec6.dropna()
dec6

,positive,negative,totalTestResults,recovered,death
0,35720.0,1042056.0,1077776.0,7165.0,143.0
1,269877.0,1421126.0,1645041.0,168387.0,3889.0
2,170924.0,1614979.0,1763150.0,149490.0,2660.0
4,364276.0,2018813.0,2370499.0,56382.0,6950.0
6,260581.0,1608829.0,3478160.0,13941.0,2724.0
7,127715.0,3294383.0,3422098.0,9800.0,5146.0
8,23136.0,711497.0,734633.0,16665.0,697.0
9,39912.0,400854.0,778298.0,18851.0,793.0
12,7004.0,79571.0,86575.0,6056.0,113.0
13,18842.0,295153.0,701776.0,11958.0,262.0


In [ ]:
dec6.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/COVID19.csv', index = False)

In [ ]:
dec6.to_csv('/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/COVID19.csv', index = False)

In [3]:
U = np.array([1,2,3])
X = np.array([[2,3,4,5], [5,6,7,8], [1,4,5,6]])

print(X)
print(X.T)

#(X[:, U] * X[:, U])

[[2 3 4 5]
 [5 6 7 8]
 [1 4 5 6]]
[[2 5 1]
 [3 6 4]
 [4 7 5]
 [5 8 6]]


In [4]:
np.dot(X, X.T)

array([[ 54,  96,  64],
       [ 96, 174, 112],
       [ 64, 112,  78]])

In [5]:
repmat(np.mat(np.diag(X)).T, 1, 5)

matrix([[2, 2, 2, 2, 2],
        [6, 6, 6, 6, 6],
        [5, 5, 5, 5, 5]])

In [6]:
repmat(np.mat(np.diag(X)), 5, 1)

matrix([[2, 6, 5],
        [2, 6, 5],
        [2, 6, 5],
        [2, 6, 5],
        [2, 6, 5]])

In [20]:
int(np.ceil(5 * np.random.rand()))

3

In [13]:
len(range(0,5))

5

In [22]:
np.array(range(5))

array([0, 1, 2, 3, 4])

In [ ]:
t = np.diag(X)
print(t)
np.where(t != 2)[0]

In [ ]:
repmat(np.diag(X), 5, 1)

In [ ]:
X[:,U]*X[:,U]

In [ ]:
ind_t = 2
ind_t[0]

In [ ]:
X

In [ ]:
X[[ind_t],:] * X[ind_t]

In [ ]:
np(X + X[ind_t])

In [ ]:
np.lib.scimath.sqrt(X + X[ind_t])

In [ ]:
Kt = np.array([[1,2], [5,6], [8,9]])
Kt = pd.DataFrame(Kt)
Kt.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/small.csv', index = False)

In [ ]:
e = np.ones((4, 1))
e

In [ ]:
CtXtXC = [2,3]
S = [3,4]
XCtX = [1,1]
SST = 5
J = 5
g = (np.dot(CtXtXC, S) - XCtX) / (SST / J)
g

In [ ]:
a = np.arange(10)
a.clip(min=5)

In [ ]:
a = range(2)
print(a)

In [ ]:
X = np.matrix([[1, 2, 3],
    [4, 5, 6], [7,8,9]])
U = [0,1,2]
print(U)
print(X)
print(X[:, U])

In [ ]:
np.sum(np.multiply(X[:, U],X[:, U]))

In [ ]:
[int(np.ceil(len(U) * np.random.rand()))]

In [ ]:
noc = 3
I = [1,2]
i = [3]
j = 3

In [ ]:
len(I)

In [ ]:
Kt = np.array([[1,2], [5,6], [8,9]])
print(Kt)
i = [1]
[I,J] = Kt.shape
print(I)
print(J)

In [ ]:
index = np.array(range(J))
index[i] = -1
index

ind_t = i
sum_dist = np.zeros((1, J), np.complex128)
sum_dist

In [ ]:
np.random.random((noc, len(U)))

In [ ]:
result = Kt**2
result

In [ ]:
Kt2 = np.sum(Kt**2, axis=0)
print(Kt2)


In [ ]:
Kq = np.dot(Kt[:, i[0]], Kt)
Kq

In [ ]:
np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[i[0]])

In [ ]:
sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[i[0]])
sum_dist

In [ ]:
index[i[0]] = i[0]
index

In [ ]:
i = i[1:]
i

In [ ]:
index

In [ ]:
t = np.where(index != 3)[0]
t

In [ ]:
Kt[:, ind_t].T

In [ ]:
Kq = np.dot(Kt[:, ind_t].T, Kt)
Kq

In [ ]:
np.dot(np.ones((noc, 1)), np.mat(np.sum(S, axis=0)))

In [ ]:
np.ones((noc, 1))

In [ ]:
np.sum(S, axis=0)

In [ ]:
np.mat(np.sum(S, axis=0))

In [ ]:
ind

In [ ]:
S

In [ ]:
np.sum(S, axis=0)

In [ ]:
np.sum(S, axis=1)

In [ ]:
l = [1,9,3,4,5,6]
max(zip(range(len(l)),l), key = lambda x:x[1])

In [ ]:
sum_dist

In [ ]:
index

In [ ]:
t = np.where(index != -1)[0]
t

In [ ]:
sum_dist = np.zeros((1,5), np.complex128)
sum_dist[0][0] += 2
sum_dist[0][4] += 5

print(sum_dist)
sum_dist[:,[2,3,4]][0].real


In [ ]:
index = np.array([0,2,-1,4,5])

In [ ]:
i = 2
i

In [ ]:
t = np.where(index != -1)[0]

In [ ]:
sum_dist

In [ ]:
max(sum_dist[:,[0,1,3,4]][0].real)

In [ ]:
vals

In [ ]:
print(pd.__version__)

In [23]:
from scipy.sparse import csr_matrix

In [28]:
j = [0,1,2]

In [29]:
i = [1,2,0]

In [32]:
csr_matrix((np.ones(len(i)), (i, j)), shape=(5, 3)).todense()

matrix([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [38]:
U

array([1, 2, 3])

In [40]:
np.log(np.random.random((3, len(U))))

array([[-1.70798884, -2.0413184 , -0.26169036],
       [-0.57663275, -2.78983353, -1.67092544],
       [-1.73874978, -0.71992443, -0.2908878 ]])

In [54]:
X = np.ones((3, 5))*2
S = np.ones((3, 5))*8
g = S*X
g

array([[16., 16., 16., 16., 16.],
       [16., 16., 16., 16., 16.],
       [16., 16., 16., 16., 16.]])

In [ ]:
np.sum(XCtX.A * S.A)

In [17]:
e = np.ones((5, 1))
e is array

NameError: name 'array' is not defined

In [61]:
r = np.sum(g * g, axis=0)
r.shape

(5,)

In [63]:
e * np.sum(g * g, axis=0)

array([[768., 768., 768., 768., 768.],
       [768., 768., 768., 768., 768.],
       [768., 768., 768., 768., 768.],
       [768., 768., 768., 768., 768.],
       [768., 768., 768., 768., 768.]])

In [4]:
dec6_wo_st = dec6[['positive','negative','totalTestResults','recovered', 'death']]
dec6_wo_st = dec6_wo_st.to_numpy()
dec6_wo_st

array([[3.5720000e+04, 1.0420560e+06, 1.0777760e+06, 7.1650000e+03,
        1.4300000e+02],
       [2.6987700e+05, 1.4211260e+06, 1.6450410e+06, 1.6838700e+05,
        3.8890000e+03],
       [1.7092400e+05, 1.6149790e+06, 1.7631500e+06, 1.4949000e+05,
        2.6600000e+03],
       [3.6427600e+05, 2.0188130e+06, 2.3704990e+06, 5.6382000e+04,
        6.9500000e+03],
       [2.6058100e+05, 1.6088290e+06, 3.4781600e+06, 1.3941000e+04,
        2.7240000e+03],
       [1.2771500e+05, 3.2943830e+06, 3.4220980e+06, 9.8000000e+03,
        5.1460000e+03],
       [2.3136000e+04, 7.1149700e+05, 7.3463300e+05, 1.6665000e+04,
        6.9700000e+02],
       [3.9912000e+04, 4.0085400e+05, 7.7829800e+05, 1.8851000e+04,
        7.9300000e+02],
       [7.0040000e+03, 7.9571000e+04, 8.6575000e+04, 6.0560000e+03,
        1.1300000e+02],
       [1.8842000e+04, 2.9515300e+05, 7.0177600e+05, 1.1958000e+04,
        2.6200000e+02],
       [2.1339000e+05, 8.8519900e+05, 1.0985890e+06, 1.5906300e+05,
        2.68

In [5]:
XC, S, C, SSE, varexpl = PCHA(dec6_wo_st, noc=3, I = None, delta=0.1)

In [6]:
XC

matrix([[3.62547274e+03, 1.07775608e+06, 1.04205396e+06],
        [8.54697280e+04, 1.64501650e+06, 1.42112621e+06],
        [7.54784891e+04, 1.76312050e+06, 1.61497740e+06],
        [3.14652900e+04, 2.37046302e+06, 2.01881307e+06],
        [8.28697659e+03, 3.47809970e+06, 1.60882963e+06],
        [7.45418651e+03, 3.42203515e+06, 3.29437676e+06],
        [8.61613603e+03, 7.34619391e+05, 7.11495592e+05],
        [9.74864628e+03, 7.78284004e+05, 4.00853778e+05],
        [3.06035609e+03, 8.65735133e+04, 7.95709027e+04],
        [6.06248481e+03, 7.01762910e+05, 2.95152778e+05],
        [8.02376885e+04, 1.09857354e+06, 8.85199763e+05],
        [2.18117823e+04, 4.90380497e+05, 3.98027520e+05],
        [1.14265640e+05, 4.57713271e+06, 1.92826391e+06],
        [2.69310201e+03, 8.45174223e+05, 6.76891653e+05],
        [1.60024147e+04, 2.73195753e+06, 2.56853949e+06],
        [1.03940040e+05, 3.60801993e+06, 3.35695529e+06],
        [8.97614901e+04, 8.97008512e+06, 3.25072695e+06],
        [6.799

In [7]:
def furthest_sum(K, noc, i, exclude=[]):
    
    def max_ind_val(l):
        return max(zip(range(len(l)), l), key=lambda x: x[1])
    
    I, J = K.shape
    index = np.array(range(J))
    index[exclude] = 0
    index[i] = -1
    ind_t = i
    sum_dist = np.zeros((1, J), np.complex128)

    if J > noc * I:
        Kt = K
        Kt2 = np.sum(Kt**2, axis=0)
        for k in range(1, noc + 11):
            if k > noc - 1:
                Kq = np.dot(Kt[:, i[0]], Kt)
                sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[i[0]])
                index[i[0]] = i[0]
                i = i[1:]
            t = np.where(index != -1)[0]
            Kq = np.dot(Kt[:, ind_t].T, Kt)
            sum_dist += np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[ind_t])
            ind, val = max_ind_val(sum_dist[:, t][0].real)
            ind_t = t[ind]
            i.append(ind_t)
            index[ind_t] = -1
    else:
        if I != J or np.sum(K - K.T) != 0:  # Generate kernel if K not one
            Kt = K
            K = np.dot(Kt.T, Kt)
            K = np.lib.scimath.sqrt(
                repmat(np.diag(K), J, 1) - 2 * K + \
                repmat(np.mat(np.diag(K)).T, 1, J)
            )

        Kt2 = np.diag(K)  # Horizontal
        for k in range(1, noc + 11):
            if k > noc - 1:
                sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * K[i[0], :] + Kt2[i[0]])
                index[i[0]] = i[0]
                i = i[1:]
            t = np.where(index != -1)[0]
            sum_dist += np.lib.scimath.sqrt(Kt2 - 2 * K[ind_t, :] + Kt2[ind_t])
            ind, val = max_ind_val(sum_dist[:, t][0].real)
            ind_t = t[ind]
            i.append(ind_t)
            index[ind_t] = -1

    return i


In [8]:
"""Principal Convex Hull Analysis (PCHA) / Archetypal Analysis."""

import numpy as np
from scipy.sparse import csr_matrix
from datetime import datetime as dt
import time

#from py_pcha.furthest_sum import furthest_sum


def PCHAm(X, noc, I=None, U=None, delta=0, verbose=False, conv_crit=1E-6, maxiter=500):
    """Return archetypes of dataset.
    Note: Commonly data is formatted to have shape (examples, dimensions).
    This function takes input and returns output of the transposed shape,
    (dimensions, examples).
    Parameters
    ----------
    X : numpy.2darray
        Data matrix in which to find archetypes
    noc : int
        Number of archetypes to find
    I : 1d-array
        Entries of X to use for dictionary in C (optional)
    U : 1d-array
        Entries of X to model in S (optional)
    Output
    ------
    XC : numpy.2darray
        I x noc feature matrix (i.e. XC=X[:,I]*C forming the archetypes)
    S : numpy.2darray
        noc x length(U) matrix, S>=0 |S_j|_1=1
    C : numpy.2darray
        noc x length(U) matrix, S>=0 |S_j|_1=1
    SSE : float
        Sum of Squared Errors
    varexlp : float
        Percent variation explained by the model
    """
    def S_update(S, XCtX, CtXtXC, muS, SST, SSE, niter):
        """Update S for one iteration of the algorithm."""
        noc, J = S.shape
        e = np.ones((noc, 1))
        #print(e.ndim)
        for k in range(niter):
            SSE_old = SSE
            g = (np.dot(CtXtXC, S) - XCtX) / (SST / J)
            #print(np.sum(g.A * S.A, axis=0))
            g = g - e * np.sum(g.A * S.A, axis=0)
            #print(g)

            S_old = S
            while True:
                S = (S_old - g * muS).clip(min=0)
                S = S / np.dot(e, np.sum(S, axis=0))
                SSt = S * S.T
                SSE = SST - 2 * np.sum(XCtX.A * S.A) + np.sum(CtXtXC.A * SSt.A)
                if SSE <= SSE_old * (1 + 1e-9):
                    muS = muS * 1.2
                    break
                else:
                    muS = muS / 2

        return S, SSE, muS, SSt

    def C_update(X, XSt, XC, SSt, C, delta, muC, mualpha, SST, SSE, niter=1):
        """Update C for one iteration of the algorithm."""
        J, nos = C.shape

        if delta != 0:
            alphaC = np.sum(C, axis=0).A[0]
            C = np.dot(C, np.diag(1 / alphaC))

        e = np.ones((J, 1))
        XtXSt = np.dot(X.T, XSt)

        for k in range(niter):

            # Update C
            SSE_old = SSE
            g = (np.dot(X.T, np.dot(XC, SSt)) - XtXSt) / SST

            if delta != 0:
                g = np.dot(g, np.diag(alphaC))
            g = g.A - e * np.sum(g.A * C.A, axis=0)

            C_old = C
            while True:
                C = (C_old - muC * g).clip(min=0)
                nC = np.sum(C, axis=0) + np.finfo(float).eps
                C = np.dot(C, np.diag(1 / nC.A[0]))

                if delta != 0:
                    Ct = C * np.diag(alphaC)
                else:
                    Ct = C

                XC = np.dot(X, Ct)
                CtXtXC = np.dot(XC.T, XC)
                SSE = SST - 2 * np.sum(XC.A * XSt.A) + np.sum(CtXtXC.A * SSt.A)

                if SSE <= SSE_old * (1 + 1e-9):
                    muC = muC * 1.2
                    break
                else:
                    muC = muC / 2

            # Update alphaC
            SSE_old = SSE
            if delta != 0:
                g = (np.diag(CtXtXC * SSt).T / alphaC - np.sum(C.A * XtXSt.A)) / (SST * J)
                alphaC_old = alphaC
                while True:
                    alphaC = alphaC_old - mualpha * g
                    alphaC[alphaC < 1 - delta] = 1 - delta
                    alphaC[alphaC > 1 + delta] = 1 + delta

                    XCt = np.dot(XC, np.diag(alphaC / alphaC_old))
                    CtXtXC = np.dot(XCt.T, XCt)
                    SSE = SST - 2 * np.sum(XCt.A * XSt.A) + np.sum(CtXtXC.A * SSt.A)

                    if SSE <= SSE_old * (1 + 1e-9):
                        mualpha = mualpha * 1.2
                        XC = XCt
                        break
                    else:
                        mualpha = mualpha / 2

        if delta != 0:
            C = C * np.diag(alphaC)

        return C, SSE, muC, mualpha, CtXtXC, XC

    N, M = X.shape
    

    if I is None:
        I = range(M)
    if U is None:
        U = range(M)

    SST = np.sum(X[:, U] * X[:, U])
    print(SST)

    # Initialize C
    try:
        i = furthest_sum(X[:, I], noc, [int(np.ceil(len(I) * np.random.rand()))])
    except IndexError:
        class InitializationException(Exception): pass
        raise InitializationException("Initialization does not converge. Too few examples in dataset.")

    #print(i)
        
    j = range(noc)
    C = csr_matrix((np.ones(len(i)), (i, j)), shape=(len(I), noc)).todense()
    #print(C)
    
    XC = np.dot(X[:, I], C)
    #print(X[:,I])

    muS, muC, mualpha = 1, 1, 1

    # Initialise S
    XCtX = np.dot(XC.T, X[:, U])
    CtXtXC = np.dot(XC.T, XC)
    S = -np.log(np.random.random((noc, len(U))))
    S = S / np.dot(np.ones((noc, 1)), np.mat(np.sum(S, axis=0)))
    SSt = np.dot(S, S.T)
    SSE = SST - 2 * np.sum(XCtX.A * S.A) + np.sum(CtXtXC.A * SSt.A)
    S, SSE, muS, SSt = S_update(S, XCtX, CtXtXC, muS, SST, SSE, 25)

    # Set PCHA parameters
    iter_ = 0
    dSSE = np.inf
    t1 = dt.now()
    varexpl = (SST - SSE) / SST

    if verbose:
        print('\nPrincipal Convex Hull Analysis / Archetypal Analysis')
        print('A ' + str(noc) + ' component model will be fitted')
        print('To stop algorithm press control C\n')

    dheader = '%10s | %10s | %10s | %10s | %10s | %10s | %10s | %10s' % ('Iteration', 'Expl. var.', 'Cost func.', 'Delta SSEf.', 'muC', 'mualpha', 'muS', ' Time(s)   ')
    dline = '-----------+------------+------------+-------------+------------+------------+------------+------------+'

    while np.abs(dSSE) >= conv_crit * np.abs(SSE) and iter_ < maxiter and varexpl < 0.9999:
        if verbose and iter_ % 100 == 0:
            print(dline)
            print(dheader)
            print(dline)
        told = t1
        iter_ += 1
        SSE_old = SSE

        # C (and alpha) update
        XSt = np.dot(X[:, U], S.T)
        C, SSE, muC, mualpha, CtXtXC, XC = C_update(
            X[:, I], XSt, XC, SSt, C, delta, muC, mualpha, SST, SSE, 10
        )
        #print(C)

        # S update
        XCtX = np.dot(XC.T, X[:, U])
        S, SSE, muS, SSt = S_update(
            S, XCtX, CtXtXC, muS, SST, SSE, 10
        )

        # Evaluate and display iteration
        dSSE = SSE_old - SSE
        print(dSSE)
        t1 = dt.now()
        if iter_ % 1 == 0:
            time.sleep(0.000001)
            varexpl = (SST - SSE) / SST
            if verbose:
                print('%10.0f | %10.4f | %10.4e | %10.4e | %10.4e | %10.4e | %10.4e | %10.4f \n' % (iter_, varexpl, SSE, dSSE/np.abs(SSE), muC, mualpha, muS, (t1-told).seconds))

    # Display final iteration
    varexpl = (SST - SSE) / SST
    if verbose:
        print(dline)
        print(dline)
        print('%10.0f | %10.4f | %10.4e | %10.4e | %10.4e | %10.4e | %10.4e | %10.4f \n' % (iter_, varexpl, SSE, dSSE/np.abs(SSE), muC, mualpha, muS, (t1-told).seconds))

    #print(S)
    #print(C)
    #print(XC)
        
    # Sort components according to importance
    ind, vals = zip(
        *sorted(enumerate(np.sum(S, axis=1)), key=lambda x: x[0], reverse=1)
    )
    S = S[ind, :]
    C = C[:, ind]
    XC = XC[:, ind]

    return XC, S, C, SSE, varexpl

In [9]:
XC2, S2, C2, SSE2, varexpl2 = PCHAm(dec6_wo_st, noc=3, I = None, delta=0)

5424894852539.0
5370130220903.5
5313150647184.5
5247144364941.5
5171021251326.75
5083838629812.25
4984810107357.25
4873501144858.0
4751320465372.75
4646259977230.25
1013087594582.75
4411807257956.25
4237446404335.0
4054063439681.0
3864494504441.5
3672125935448.0
3480730093016.5
3294254933740.75
3116592750570.5
2951355350482.0
2801676490328.75
1952649731572.75
2459157526383.5
2254248986132.5
2087904963014.25
1958046106530.5
1861750311165.5
1794820625128.75
1751510002431.75
1724855141346.75
1707890110874.0
1695258619321.5
984047171525.25
1475110354858.25
1378828673945.5
1356499193357.0
1337070725731.75
1320130965392.75
1306275898349.0
1295841697888.25
1288891631593.5
1285020596973.5
1283772119147.25
281361973868.0
1193748380990.25
1154213475231.25
1150979172541.5
1149010773296.0
1148627787016.5
1148627786530.5
1148627786515.5
1148627786513.25
1148627786513.25
1148627786516.5
64788194570.0
1128960186420.25
1126964684467.0
1124062559446.0
1124051578000.25
1124050920509.5
1124050843578.25
1

In [28]:
varexpl2

0.9993767918374076

In [ ]:
ans = 0

for i in range(len(XC)):
    a = XC[i] - XC2[i]
    ans = ans + a
    
print(ans)

In [ ]:
ans = 0

for i in range(len(S)):
    a = S[i] - S2[i]
    ans = ans + a
    
print(ans)

In [ ]:
ans = 0

for i in range(len(C)):
    a = C[i] - C2[i]
    ans = ans + a
    
print(ans)